In [86]:
from bs4 import BeautifulSoup as soup
import requests
from datetime import datetime
import pandas as pd

In [87]:
data = requests.get(f'https://thestandard.co/page/2/?s=7-eleven&search')
dd = soup(data.content,'html.parser')

In [88]:
title_list = []
urls = []
author_list = []
desc_list = []
published_date = []
retrieved_date  = []

In [89]:
for news in dd.findAll('div', attrs={'class' : 'col-md-8'}):
    title_list.append(news.find('h3',{'class':'news-title'}).find('a').text.replace('\n',''))
    urls.append(news.find('a', href=True)['href'])

In [90]:
for link in urls:
    page = requests.get(link)
    b = soup(page.content,'html.parser')
    for a in b.findAll('div', attrs={'class' : 'meta-author'}):
        author_list.append(a.find('a').text)
        
    for n in b.findAll('div', attrs={'class' : 'entry-content'}):
        des = n.text.replace('\n','').replace('\xa0','')
        desc_list.append(des)

    for date in b.findAll('div', attrs={'class' : 'meta-date'}):
        date_p = date.find('span').get_text()
        date_p = datetime.strptime(date_p, '%d.%m.%Y').strftime("%Y-%m-%d")
        published_date.append(date_p)
        
    r = (datetime.now()).strftime("%Y-%m-%d")
    retrieved_date.append(r)

In [91]:
table = pd.DataFrame([title_list, published_date, retrieved_date, author_list, desc_list, urls]).transpose()
table.columns = ['title','publish_date','retrieve_date','author','description','source_url']
table.head()

,title,publish_date,retrieve_date,author,description,source_url
0,บริษัทแม่ 7-Eleven ในญี่ปุ่น ถูกนักลงทุนสถาบัน...,2022-01-27,2022-11-15,ถนัดกิจ จันกิเสน,Seven & i Holdings กลุ่มค้าปลีกรายใหญ่ที่สุดขอ...,https://thestandard.co/7-eleven-japan-pressure...
1,กลุ่มค้าปลีกรุกเจาะออนไลน์โตก้าวกระโดด 7-Eleve...,2021-12-24,2022-11-15,THE STANDARD WEALTH,ปี 2564 ที่ผ่านมา โควิดมีทั้งวิกฤตและโอกาส แต่...,https://thestandard.co/retail-group-penetrates...
2,ใช้คนเดินส่งธรรมดาไป! 7-Eleven ในสหรัฐฯ จับมือ...,2021-12-02,2022-11-15,ถนัดกิจ จันกิเสน,แม้วันนี้การบริการเดลิเวอรีจาก 7-Eleven ในบ้าน...,https://thestandard.co/7-eleven-nuro-autonomou...
3,‘ฟูซิลลี่ผัดขี้เมาไก่’ เมนูจัดจ้านจาก ‘เจ๊ไฝ’ ...,2021-10-30,2022-11-15,วรรษชล คัวดรี้,นอกจาก เจ๊ไฝ-สุภิญญา จันสุตะ จากร้านเจ๊ไฝ สตรี...,https://thestandard.co/jay-fai-7-eleven/
4,"‘อินเดีย’ ประเทศที่มีกำลังซื้อ 1,400 ล้านคน กำ...",2021-10-08,2022-11-15,ถนัดกิจ จันกิเสน,วันเสาร์ที่ 9 ตุลาคมนี้ จะเป็นวันแรกที่ 7-Elev...,https://thestandard.co/7-eleven-india-first-br...


In [92]:
table.to_csv('04scrap_7eleven_TheStandard_news.csv',encoding='utf-8-sig')